In [67]:
import pandas as pd
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
from tsfresh.feature_extraction import extract_features, extract_feature_dynamics
from tsfresh.feature_selection import select_features
from tsfresh.feature_extraction.settings import MinimalFCParameters

from tsfresh.feature_extraction.gen_example_timeseries_data import gen_example_timeseries_data
from IPython.display import display
from json import dumps

# Introduction

## Topics covered
1. tsfresh feature extraction recap
2. Extracting feature dynamics with tsfresh
3. Interpreting Feature dynamics
4. Deriving a custom set of feature calculators
5. Generating new input time series 
6. Generating a pdf document from the extracted feature dynamics


## Load Example Data
In this workbook we will considering an two datasets: 
1. [Robot Execution Failures Dataset](https://archive.ics.uci.edu/ml/datasets/Robot+Execution+Failures) to demonstrate how the extraction of feature dynamics. works. 
2. Then we will use another exemple dataset latter which was manually generated.

The data set documents 88 robot executions (each has a unique id between 1 and 88), which is a subset of the Robot Execution Failures Data Set. 
For the purpose of simplicity we are only differentiating between successfull and failed executions (`y`).

For each execution 15 force (`F`) and torque (`T`) samples are given, which were measured at regular time intervals for the spatial dimensions x, y, and z. 
Therefore each row of the data frame references a specific execution (`id`), a time index (`index`) and documents the respective measurements of 6 sensors (`F_x, F_y, F_z, T_x, T_y, T_z`).

The Robot dataset is unbalanced and for the purposes of this tutorial we will not ignore the issue of undersampling as this is merely a toy example to demonstrate how the new code works.

In [68]:
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()
display(timeseries.head())

,id,time,F_x,F_y,F_z,T_x,T_y,T_z
0,1,0,-1,-1,63,-3,-1,0
1,1,1,0,0,62,-3,-1,0
2,1,2,-1,-1,61,-3,0,0
3,1,3,-1,-1,63,-2,-1,0
4,1,4,-1,-1,63,-3,-1,0


# Extract features from the Time Series
Let us start by recapping how a simple set of time series features (mean, median, max, variance, ...) are calculated using tsfresh.

In [69]:
extracted_features = extract_features(timeseries, 
                                    column_id="id", 
                                    column_sort="time", 
                                    default_fc_parameters=MinimalFCParameters())
display(extracted_features.head())

Feature Extraction: 100%|██████████| 10/10 [00:05<00:00,  1.95it/s]


,F_x__sum_values,F_x__median,F_x__mean,F_x__length,F_x__standard_deviation,F_x__variance,F_x__maximum,F_x__minimum,F_y__sum_values,F_y__median,...,T_y__maximum,T_y__minimum,T_z__sum_values,T_z__median,T_z__mean,T_z__length,T_z__standard_deviation,T_z__variance,T_z__maximum,T_z__minimum
1,-14.0,-1.0,-0.933333,15.0,0.249444,0.062222,0.0,-1.0,-13.0,-1.0,...,0.0,-1.0,0.0,0.0,0.000000,15.0,0.000000,0.000000,0.0,0.0
2,-13.0,-1.0,-0.866667,15.0,0.956847,0.915556,0.0,-3.0,-10.0,-1.0,...,4.0,-5.0,-4.0,0.0,-0.266667,15.0,0.442217,0.195556,0.0,-1.0
3,-10.0,-1.0,-0.666667,15.0,0.596285,0.355556,1.0,-1.0,-8.0,0.0,...,1.0,-5.0,-4.0,0.0,-0.266667,15.0,0.442217,0.195556,0.0,-1.0
4,-6.0,0.0,-0.400000,15.0,0.952190,0.906667,1.0,-2.0,2.0,1.0,...,4.0,-6.0,-5.0,0.0,-0.333333,15.0,0.596285,0.355556,1.0,-1.0
5,-9.0,-1.0,-0.600000,15.0,0.879394,0.773333,2.0,-2.0,-4.0,0.0,...,3.0,-5.0,-2.0,0.0,-0.133333,15.0,0.618241,0.382222,1.0,-1.0


And when it comes to feature selection it is as simple as:

In [70]:
## Typical feature extraction
selected_features = select_features(extracted_features,y)
display(selected_features.head())

,T_y__standard_deviation,T_y__variance,F_z__standard_deviation,F_z__variance,F_x__standard_deviation,F_x__variance,T_x__standard_deviation,T_x__variance,F_y__variance,F_y__standard_deviation,...,F_z__mean,F_z__median,F_y__maximum,F_x__minimum,T_x__minimum,F_x__maximum,T_y__minimum,T_z__maximum,T_z__minimum,F_z__maximum
1,0.471405,0.222222,1.203698,1.448889,0.249444,0.062222,0.339935,0.115556,0.115556,0.339935,...,62.533333,63.0,0.0,-1.0,-3.0,0.0,-1.0,0.0,0.0,64.0
2,2.054805,4.222222,4.333846,18.782222,0.956847,0.915556,3.422799,11.715556,4.622222,2.149935,...,62.133333,63.0,3.0,-3.0,-10.0,0.0,-5.0,0.0,-1.0,70.0
3,1.768867,3.128889,4.616877,21.315556,0.596285,0.355556,2.633122,6.933333,2.382222,1.543445,...,61.133333,61.0,2.0,-1.0,-7.0,1.0,-5.0,0.0,-1.0,68.0
4,2.669998,7.128889,3.833188,14.693333,0.952190,0.906667,3.525148,12.426667,3.982222,1.995551,...,62.200000,63.0,5.0,-2.0,-15.0,1.0,-6.0,1.0,-1.0,70.0
5,2.039608,4.160000,4.841487,23.440000,0.879394,0.773333,2.756810,7.600000,2.995556,1.730767,...,60.600000,59.0,3.0,-2.0,-12.0,2.0,-5.0,1.0,-1.0,73.0


Now lets take it to the next level!

# Introducing Feature Dynamics

# How does the extraction of feature dynamics work?
Should we find that these features themselves are not sufficiently informative we can try an alternative approach: the extraction of feature dynamics! 
This is accomplished by the function `extract_feature_dynamics` in `tsfresh.feature_extraction.extraction`.

In principle this works as such:

1. The input time *X* series is windowed and the chosen set of *N* features are extracted. This returns a new matrix *M* where each column represents a particular **feature time series**.

2. For each feature in the resulting output *M*, step 1 is repeated and for the chosen feature time series. Each new column generated can be referred to as a  **feature-dynamic(s)**
    
3. Repeat for each column in *M*.

## Differences between `extract_features` & `extract_feature_dynamics`
`extract_feature_dynamics` shares most of the same parameters as `extract_features`, but the key differences are:
* `window_length` - this specifies the length of the time series window from which the first set of features is extracted.
* `feature_timeseries_fc_parameters` - this specifies the type of feature calculator dictionary object will be used to calculate the **feature timeseries** **from our input**.
* `feature_dynamics_fc_parameters` - this specifies the type of feature calculator dictionary object will be used to calculate the **feature dynamics** from our **feature time series**.
* `feature_timeseries_kind_to_fc_parameters` - this specifies the custom feature calculator to calculate **feature timeseries**.
* `feature_dynamics_kind_to_fc_parameters` - - this specifies the custom feature calculator to calculate **feature dynamics**.

## Computational challenges ##
A major caveat associated with this approach is that extracting feature dynamics *can* lead to an exponential number of columns being generated. 

##### Simple Example: 
If the input has just 1 time series (1 column) and we extract ***N*** features, then ***N*** feature dynamics this 
will result in ***1*** x ***N*** x ***N*** columns in total!

This *can* be highly computationally intensive and users should beware of.
It is also worth noting that computational time/effort is affected by: window length, number of processors/ parallelisation, which features are computed amongst other factors.
For this reason a high performance computer with multiple was used to develop a proof of concept implementation of this feature-engineering algorithm.

##### Recommendation
Therefore, when testing code we *strongly* recommend using the feature set specified by `MinimalFCParameters()` located in `tsfresh.feature_extraction.settings`.

Below the function is demonstrated on the same robot executaion failures dataset.

In [71]:

extracted_sub_features = extract_feature_dynamics(timeseries_container=timeseries,
                                    window_length=100,  # window size arbrarily chosen
                                    column_id="id",
                                    column_sort="time",
                                    feature_timeseries_fc_parameters=MinimalFCParameters(),
                                    feature_dynamics_fc_parameters=MinimalFCParameters())
display(extracted_sub_features.head())                                   

Feature Extraction: 100%|██████████| 10/10 [00:10<00:00,  1.02s/it]


,F_x||length__sum_values,F_x||length__median,F_x||length__mean,F_x||length__length,F_x||length__standard_deviation,F_x||length__variance,F_x||length__maximum,F_x||length__minimum,F_x||maximum__sum_values,F_x||maximum__median,...,T_z||sum_values__maximum,T_z||sum_values__minimum,T_z||variance__sum_values,T_z||variance__median,T_z||variance__mean,T_z||variance__length,T_z||variance__standard_deviation,T_z||variance__variance,T_z||variance__maximum,T_z||variance__minimum
1,15.0,15.0,15.0,1.0,0.0,0.0,15.0,15.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.000000
2,15.0,15.0,15.0,1.0,0.0,0.0,15.0,15.0,0.0,0.0,...,-4.0,-4.0,0.195556,0.195556,0.195556,1.0,0.0,0.0,0.195556,0.195556
3,15.0,15.0,15.0,1.0,0.0,0.0,15.0,15.0,1.0,1.0,...,-4.0,-4.0,0.195556,0.195556,0.195556,1.0,0.0,0.0,0.195556,0.195556
4,15.0,15.0,15.0,1.0,0.0,0.0,15.0,15.0,1.0,1.0,...,-5.0,-5.0,0.355556,0.355556,0.355556,1.0,0.0,0.0,0.355556,0.355556
5,15.0,15.0,15.0,1.0,0.0,0.0,15.0,15.0,2.0,2.0,...,-2.0,-2.0,0.382222,0.382222,0.382222,1.0,0.0,0.0,0.382222,0.382222


The full feature feature engineering pipeline (including the selection of relevant features) is outlined in the following diagram:

<center>
    <img src="./features_on_features_diagram.png" width = 600, height=400 /> 
</center>

More detail on how this approach has been used on extremely long time series can be found in the IEEE paper: ["Data Mining on Extremely Long Time Series"](https://ieeexplore.ieee.org/document/9679945).

# Interpreting the results

As can be seen, running `extract_features_on_sub_features` results in significantly more columns

## Decomposing the column names
`"F_x||length__sum_values"`

This is demonstrated below

In [73]:
from tsfresh.feature_extraction.gen_features_dicts_function import derive_features_dictionaries

## Take a subset of the columns to demonstrate (reduce size of output)
sub_feature_names = extracted_sub_features.columns.tolist()[:120]
f,ff = derive_features_dictionaries(sub_feature_names)

print("The set of features calculated on the original time series:\n")
print(dumps(f,sort_keys=True, indent=4))


The set of features calculated on the original time series:

{
    "F_x": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null,
        "variance": null
    },
    "F_y": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null
    }
}


`F_x` is the time series and `length, maximum, mean..., variance` are the features calculated.

`"length" : "null"` means that the length feature does.

For more info on feature naming see [Feature Calculator Naming](https://tsfresh.readthedocs.io/en/latest/text/feature_calculation.html)

# Understanding Features on Features

In [79]:
print("\nAn example of some feature-dynamics generated on the feature time-series:\n")
print(dumps(ff,sort_keys=True, indent=4)[:479])


An example of some feature-dynamics generated on the feature time-series:

{
    "F_x||length": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null,
        "variance": null
    },
    "F_x||maximum": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null,
        "variance": null
    },
  


The `F_x||length` is the feature time series:
i.e. length calculated from F_x.



### Select the most relevant Feature Dynamics
It works exactly the same as with extract_features e.g.

In [74]:
## repeat the process but now we extract feature dynamics
selected_sub_features = select_features(extracted_sub_features,y)
display(selected_sub_features.head())

,T_y||standard_deviation__mean,T_y||standard_deviation__sum_values,T_y||standard_deviation__median,T_y||variance__minimum,T_y||variance__maximum,T_y||variance__mean,T_y||variance__median,T_y||variance__sum_values,T_y||standard_deviation__minimum,T_y||standard_deviation__maximum,...,T_z||minimum__minimum,T_z||minimum__maximum,T_z||minimum__mean,T_z||minimum__median,T_z||minimum__sum_values,F_z||maximum__sum_values,F_z||maximum__median,F_z||maximum__mean,F_z||maximum__maximum,F_z||maximum__minimum
1,0.471405,0.471405,0.471405,0.222222,0.222222,0.222222,0.222222,0.222222,0.471405,0.471405,...,0.0,0.0,0.0,0.0,0.0,64.0,64.0,64.0,64.0,64.0
2,2.054805,2.054805,2.054805,4.222222,4.222222,4.222222,4.222222,4.222222,2.054805,2.054805,...,-1.0,-1.0,-1.0,-1.0,-1.0,70.0,70.0,70.0,70.0,70.0
3,1.768867,1.768867,1.768867,3.128889,3.128889,3.128889,3.128889,3.128889,1.768867,1.768867,...,-1.0,-1.0,-1.0,-1.0,-1.0,68.0,68.0,68.0,68.0,68.0
4,2.669998,2.669998,2.669998,7.128889,7.128889,7.128889,7.128889,7.128889,2.669998,2.669998,...,-1.0,-1.0,-1.0,-1.0,-1.0,70.0,70.0,70.0,70.0,70.0
5,2.039608,2.039608,2.039608,4.160000,4.160000,4.160000,4.160000,4.160000,2.039608,2.039608,...,-1.0,-1.0,-1.0,-1.0,-1.0,73.0,73.0,73.0,73.0,73.0


# Given relevant feature dynamics we can streamine the feature extraction process

The convenience of this is that once we have selected our relevant feature dynamics we can generate two custom feature caluclator dictionaries
then pass them into the `feature_timeseries_kind_to_fc_parameters` and `feature_dynamics_kind_to_fc_parameters`.

This requires the use of `derive_features_dictionaries` as with the below example:

In [78]:
## Given the chosen feature-dynamics generate dictionary represenations of them
fc1, fc2 = derive_features_dictionaries(selected_sub_features)

## now calculate only the relevant features!
extracted_v2 = extract_feature_dynamics(timeseries, 
                                        window_length=100,
                                        column_id="id", 
                                        column_sort="time",
                                        feature_timeseries_kind_to_fc_parameters=fc1, 
                                        feature_dynamics_kind_to_fc_parameters=fc2)

print(f"{extracted_v2.shape[1]} feature dynamics are calculated here vs {extracted_sub_features.shape[1]} calculated before.")

display(extracted_v2.head())


Feature Extraction: 100%|██████████| 10/10 [00:05<00:00,  1.84it/s]


120 feature dynamics are calculated here vs 384 calculated before.


,F_x||maximum__sum_values,F_x||maximum__minimum,F_x||maximum__median,F_x||maximum__mean,F_x||maximum__maximum,F_x||minimum__sum_values,F_x||minimum__median,F_x||minimum__mean,F_x||minimum__maximum,F_x||minimum__minimum,...,T_z||standard_deviation__minimum,T_z||standard_deviation__maximum,T_z||standard_deviation__mean,T_z||standard_deviation__median,T_z||standard_deviation__sum_values,T_z||variance__median,T_z||variance__maximum,T_z||variance__mean,T_z||variance__sum_values,T_z||variance__minimum
1,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,-3.0,-3.0,-3.0,-3.0,-3.0,...,0.442217,0.442217,0.442217,0.442217,0.442217,0.195556,0.195556,0.195556,0.195556,0.195556
3,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.442217,0.442217,0.442217,0.442217,0.442217,0.195556,0.195556,0.195556,0.195556,0.195556
4,1.0,1.0,1.0,1.0,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,0.596285,0.596285,0.596285,0.596285,0.596285,0.355556,0.355556,0.355556,0.355556,0.355556
5,2.0,2.0,2.0,2.0,2.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,0.618241,0.618241,0.618241,0.618241,0.618241,0.382222,0.382222,0.382222,0.382222,0.382222


# Generating new time series
In addition to extracting features or sub_features from a time series, it may be of interest to generate new time series in the original dataset, from which we will then extract features or feature dynamics.

The function `engineer_input_timeseries` helps to automate this process, by generating new time series from the original data. 
It has the option to compute first order differences and/or the (phase) differences between each of the time series in the original dataset 
See more at `tsfresh....`

## Load Example Time Series

In [ ]:
## Load in an arbitrary example
ts_example,_ = gen_example_timeseries_data(container_type="pandas")
display(ts_example.head())


,t,y1,y2,y3,measurement_id
0,1,0.0,457.0,3454.0,1
1,1,0.0,352.0,13452.0,1
2,1,0.0,3524.0,23534.0,1
3,1,345346.0,124532.0,12432.0,1
4,1,1356.0,24.0,412432.0,1


If we we wanted to calculate more time series before extracting feature dynamics - we could look at differencing each of the initial time series measurements:

In [ ]:
from tsfresh.feature_extraction import engineer_input_timeseries

## Let us find the first order differences for each of the time series in the original dataframe
new_input_ts = engineer_input_timeseries(ts_example, 
                                        column_id="measurement_id",
                                        column_sort="t",
                                        compute_differences_within_series=True, 
                                        compute_differences_between_series=False)
display(new_input_ts.head())

,y1,y2,y3,dt_y1,dt_y2,dt_y3,measurement_id,t
0,0.0,457.0,3454.0,0.0,0.0,0.0,1,1
1,0.0,352.0,13452.0,0.0,-105.0,9998.0,1,1
2,0.0,3524.0,23534.0,0.0,3172.0,10082.0,1,1
3,345346.0,124532.0,12432.0,345346.0,121008.0,-11102.0,1,1
4,1356.0,24.0,412432.0,-343990.0,-124508.0,400000.0,1,1


Here 3 new colums are created where `dt_colname` denotes the first order differencing of time series `colname`.

### How about computing the difference between different series?

In [ ]:
## now additionally compute the differences between all paris of time series in our data
ts_diff_phase_diff = engineer_input_timeseries(ts_example,
                                                column_id="measurement_id",
                                                column_sort="t",                                         
                                                compute_differences_within_series=True, 
                                                compute_differences_between_series=True)
display(ts_diff_phase_diff.head())

,y1,y2,y3,dt_y1,dt_y2,dt_y3,D_y1y2,D_y1y3,D_y2y3,measurement_id,t
0,0.0,457.0,3454.0,0.0,0.0,0.0,-457.0,-3454.0,-2997.0,1,1
1,0.0,352.0,13452.0,0.0,-105.0,9998.0,-352.0,-13452.0,-13100.0,1,1
2,0.0,3524.0,23534.0,0.0,3172.0,10082.0,-3524.0,-23534.0,-20010.0,1,1
3,345346.0,124532.0,12432.0,345346.0,121008.0,-11102.0,220814.0,332914.0,112100.0,1,1
4,1356.0,24.0,412432.0,-343990.0,-124508.0,400000.0,1332.0,-411076.0,-412408.0,1,1


In [ ]:
## Get new columns
new_cols = list(ts_diff_phase_diff.columns.difference(new_input_ts.columns))
display(ts_diff_phase_diff[new_cols].head())

,D_y1y2,D_y1y3,D_y2y3
0,-457.0,-3454.0,-2997.0
1,-352.0,-13452.0,-13100.0
2,-3524.0,-23534.0,-20010.0
3,220814.0,332914.0,112100.0
4,1332.0,-411076.0,-412408.0


Interpretation:
Now we have new columns where the naming convention `D_col1_col2` represents the result of taking `col1 - col2`, i.e. difference between two time series.

# Generate a pdf that describes features
By calling `gen_pdf_for_feature_dynamics` with a list of column names as an input, a pdf can be generated giving a description of the the feature dynamics and the parameter used to calculate them.

#### Example


<center>
<img src="./pdf_example.png" width=400 height=500 >
</center>